<a href="https://colab.research.google.com/github/mgo-city/INM707/blob/main/notebooks/INM707_Lab_09_feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 09 - Random Search

Steps:
- Create a policy class with a method to generate perturbations over the policy.
- Create a function to evaluate a policy on an environment or a list of environments.
- create a simple optimization algorithm that solves dungeon through hill climbing.


In [1]:
!pip install git+https://github.com/mgo-city/INM707.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/mgo-city/INM707.git to /tmp/pip-req-build-t542t_gc
  Running command git clone --filter=blob:none --quiet https://github.com/mgo-city/INM707.git /tmp/pip-req-build-t542t_gc
  Resolved https://github.com/mgo-city/INM707.git to commit a882bdb2d3718841b6701e2e3e7b6f6dffea52cc
  Preparing metadata (setup.py) ... done
  Created wheel for dungeon: filename=dungeon-0.0.1-py3-none-any.whl size=3907 sha256=366f4e3dfa1d8abe9b6859f7c648af4a39707f1c73b46e8cd1bd11381a54b246
  Stored in directory: /tmp/pip-ephem-wheel-cache-wp91c11n/wheels/a9/05/b3/b5a73c697d7c0143c5557f13251faac53bd330deaf9731f066
Successfully built dungeon


# Loading the dungeon

In [2]:
from dungeon.dungeon import Dungeon, IceDungeon, index_to_actions
import numpy as np
import random

import matplotlib.pyplot as plt


In [3]:
SIZE_ENVIR = 10

my_dungeon = Dungeon(SIZE_ENVIR)
my_dungeon.reset()
my_dungeon.display()

X X X X X X X X X X 
X . . . . . X . X X 
X . . . . . . . . X 
X . . . . . . . . X 
X . . . . . . . L X 
X . . X L . . . L X 
X . . . X E . . L X 
X . X . . . . . A X 
X L . . . . . . . X 
X X X X X X X X X X 



In [4]:
my_dungeon.reset()
my_dungeon.display()

X X X X X X X X X X 
X . . . . . X . X X 
X . . . . . . . . X 
X . . . . . . . . X 
X . . . . . . . L X 
X . . X L . . . L X 
X . . A X E . . L X 
X . X . . . . . . X 
X L . . . . . . . X 
X X X X X X X X X X 



In [5]:
obs, rew, done = my_dungeon.step('up')
print(obs, rew, done)
my_dungeon.display()
for obs_name, o in obs.items():
    print(obs_name)
    print(o)

{'relative_coordinates': array([0, 2]), 'surroundings': array([[0, 0, 0, 0, 0],
       [0, 0, 1, 2, 0],
       [0, 0, 0, 1, 3],
       [0, 1, 0, 0, 0],
       [2, 0, 0, 0, 0]], dtype=int8)} -6 False
X X X X X X X X X X 
X . . . . . X . X X 
X . . . . . . . . X 
X . . . . . . . . X 
X . . . . . . . L X 
X . . X L . . . L X 
X . . A X E . . L X 
X . X . . . . . . X 
X L . . . . . . . X 
X X X X X X X X X X 

relative_coordinates
[0 2]
surroundings
[[0 0 0 0 0]
 [0 0 1 2 0]
 [0 0 0 1 3]
 [0 1 0 0 0]
 [2 0 0 0 0]]


# 1 - Create a policy class


In [56]:
import numpy as np
from numpy.random import default_rng
import math

rng = default_rng()

class Policy:

    def __init__(self, size_hidden):
        super().__init__()
        
        # Declare the different layers

        # Surrounding observation layer
        self.w_1 = rng.uniform(-math.sqrt(1/25), math.sqrt(1/25), size = (25, size_hidden) )
        self.w_2 = rng.uniform(-math.sqrt(1/size_hidden), math.sqrt(1/size_hidden), size = (size_hidden, size_hidden) )
        self.b_1 = rng.uniform(-math.sqrt(1/25), math.sqrt(1/25), size = (size_hidden,) )
        self.b_2 = rng.uniform(-math.sqrt(1/size_hidden), math.sqrt(1/size_hidden), size = (size_hidden,) )

        # Coordinate observation layer
        self.w_3 = rng.uniform(-math.sqrt(1/2), math.sqrt(1/2), size = (2, size_hidden) )
        self.w_4 = rng.uniform(-math.sqrt(1/size_hidden), math.sqrt(1/size_hidden), size = (size_hidden, size_hidden) )
        self.b_3 = rng.uniform(-math.sqrt(1/size_hidden), math.sqrt(1/size_hidden), size = (size_hidden,) )
        self.b_4 = rng.uniform(-math.sqrt(1/size_hidden), math.sqrt(1/size_hidden), size = (size_hidden,) )

        # Merge layer
        self.w_5 = rng.uniform(-math.sqrt(1/(2*size_hidden)), math.sqrt(1/(2*size_hidden)), size = (2*size_hidden, size_hidden) )
        self.w_6 = rng.uniform(-math.sqrt(1/size_hidden), math.sqrt(1/size_hidden), size = (size_hidden, 4) )
        self.b_5 = rng.uniform(-math.sqrt(1/size_hidden), math.sqrt(1/size_hidden), size = (size_hidden,) )
        self.b_6 = rng.uniform(-math.sqrt(1/size_hidden), math.sqrt(1/size_hidden), size = (4,) )

        # At initialization, no perturbations
        self.theta_w_1 = np.zeros( (25, size_hidden) )
        self.theta_w_2 = np.zeros( (size_hidden, size_hidden) )
        self.theta_w_3 = np.zeros( (2, size_hidden) )
        self.theta_w_4 = np.zeros( (size_hidden, size_hidden) )        
        self.theta_w_5 = np.zeros( (2*size_hidden, size_hidden) )        
        self.theta_w_6 = np.zeros( (size_hidden, 4) )        

        self.theta_b_1 = np.zeros( (size_hidden,) )
        self.theta_b_2 = np.zeros( (size_hidden,) )
        self.theta_b_3 = np.zeros( (size_hidden,) )
        self.theta_b_4 = np.zeros( (size_hidden,) )
        self.theta_b_5 = np.zeros( (size_hidden,) )
        self.theta_b_6 = np.zeros( (4,) )

    def relu(self, x):
        return x * (x > 0)

    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))

    def select_action(self, x):

        s = x[2:]
        c = x[:2]

        # Declare how the data flows from input x to output.
        s = self.sigmoid(self.w_1.T @ s + self.theta_w_1.T @ s + self.b_1 + self.theta_b_1)
        s = self.sigmoid(self.w_2.T @ s + self.theta_w_2.T @ s + self.b_2 + self.theta_b_2)

        c = self.sigmoid(self.w_3.T @ c + self.theta_w_3.T @ c + self.b_3 + self.theta_b_3)
        c = self.sigmoid(self.w_4.T @ c + self.theta_w_4.T @ c + self.b_4 + self.theta_b_4)

        repr = np.concatenate([c, s])

        repr = self.sigmoid(self.w_5.T @ repr + self.theta_w_5.T @ repr + self.b_5 + self.theta_b_5)
        repr = self.w_6.T @ repr + self.theta_w_6.T @ repr + self.b_6 + self.theta_b_6

        output = np.argmax(repr)

        return output

    def perturbate_policy(self, scale):

        # We perturbate only one branch of the processing

        branch = rng.choice(['s', 'c', 'repr'])

        self.theta_w_1 = np.zeros(self.theta_w_1.shape)
        self.theta_w_2 = np.zeros(self.theta_w_2.shape)
        self.theta_b_1 = np.zeros(self.b_1.shape )
        self.theta_b_2 = np.zeros(self.b_2.shape )

        self.theta_w_3 = np.zeros(self.theta_w_3.shape)
        self.theta_w_4 = np.zeros(self.theta_w_4.shape)
        self.theta_b_3 = np.zeros(self.b_3.shape )
        self.theta_b_4 = np.zeros(self.b_4.shape )

        self.theta_w_5 = np.zeros(self.theta_w_5.shape)
        self.theta_w_6 = np.zeros(self.theta_w_6.shape)
        self.theta_b_5 = np.zeros(self.b_5.shape )
        self.theta_b_6 = np.zeros(self.b_6.shape )


        if branch == 's':
          self.theta_w_1 = rng.normal( scale = scale, size = self.theta_w_1.shape)
          self.theta_w_2 = rng.normal( scale = scale, size = self.theta_w_2.shape)
          self.theta_b_1 = rng.normal(scale = scale, size = self.b_1.shape )
          self.theta_b_2 = rng.normal(scale = scale, size = self.b_2.shape )

        elif branch == 'c':
          self.theta_w_3 = rng.normal( scale = scale, size = self.theta_w_3.shape)
          self.theta_w_4 = rng.normal( scale = scale, size = self.theta_w_4.shape)
          self.theta_b_3 = rng.normal(scale = scale, size = self.b_3.shape )
          self.theta_b_4 = rng.normal(scale = scale, size = self.b_4.shape )

        elif branch == 'repr':
          self.theta_w_5 = rng.normal( scale = scale, size = self.theta_w_5.shape)
          self.theta_w_6 = rng.normal( scale = scale, size = self.theta_w_6.shape)
          self.theta_b_5 = rng.normal(scale = scale, size = self.b_5.shape )
          self.theta_b_6 = rng.normal(scale = scale, size = self.b_6.shape )

        else:
          raise ValueError

    def update_policy(self):

        self.w_1 += self.theta_w_1
        self.w_2 += self.theta_w_2
        self.w_3 += self.theta_w_3
        self.w_4 += self.theta_w_4
        self.w_5 += self.theta_w_5
        self.w_6 += self.theta_w_6

        self.b_1 += self.theta_b_1
        self.b_2 += self.theta_b_2
        self.b_3 += self.theta_b_3
        self.b_4 += self.theta_b_4
        self.b_5 += self.theta_b_5
        self.b_6 += self.theta_b_6

def convert_state(state, N):

    c = state['relative_coordinates'].flatten()
    o = state['surroundings'].flatten()

    c = c / N
    o = o / 4 - 0.5

    state = np.concatenate( [c,o] )
    
    return state

# 2: Evaluate Policy on Environment


In [57]:
def run_policy_on_environment(policy, dungeon, time_limit):

  state = dungeon.reset()
  cumulative_reward = 0
  t = 0

  while t < time_limit:
    
    state = convert_state(state, dungeon.size)
    action = policy.select_action(state)
    action_name = index_to_actions[action].name 

    state, reward, done = dungeon.step(action_name)

    cumulative_reward += reward
    
    if done:
      return cumulative_reward

    t += 1

  return -10000


def evaluate_policy(policy, episodes, time_limit, N):

  returns = []
  
  for episode in range(episodes):
    dungeon = Dungeon(N)
  
    returns.append(run_policy_on_environment(policy, dungeon, time_limit))

  return sum(returns)/len(returns)

# 3 - Simple optimization loop


In [ ]:
SIZE_ENVIR = 10
N_STEPS = 100000

policy = Policy(128)

objective = evaluate_policy(policy, 20, 50, SIZE_ENVIR)
results = []

for i in range(N_STEPS):
  policy.perturbate_policy(scale = 10*(N_STEPS - i) / N_STEPS)
  new_obj = evaluate_policy(policy, 3, 100, SIZE_ENVIR)

  if new_obj > objective:

    # verify with more eval
    new_obj_check = evaluate_policy(policy, 20, 100, SIZE_ENVIR)
    if not new_obj_check > objective:
      continue
    
    policy.update_policy()
    objective = new_obj_check
    print( f"Optimization step {i}, objective {objective}")
    results.append([i, objective])


Optimization step 0, objective -648.15
Optimization step 1, objective -589.5
Optimization step 6, objective -589.25
Optimization step 16, objective -522.05


<ipython-input-56-0754afe08dc3>:51: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-x))
